## Before continuing, please select menu option:  **Cell => All output => clear**

# First... numpy

In [ ]:
import numpy as np

In [ ]:
a = np.random.randint(1,6,size=(10, 3))
a

In [ ]:
double = a * 2
double

In [ ]:
over3 = a > 3
over3

In [ ]:
a[over3]

# Pandas Series

In [ ]:
import pandas as pd

In [ ]:
d = { 'a': 11, 'b': 22, 'c':33, 'd':44, 'e':55 }
ser = pd.Series(d)
ser

In [ ]:
for i in ser:
    print(i)

In [ ]:
ser[1]

In [ ]:
ser['b']

In [ ]:
ser[1:2] # works like a list, end exclusive slicing

In [ ]:
ser['b':'d'] # works inclusive

In [ ]:
ser * 2

In [ ]:
ser

# Pandas Dataframe

In [ ]:
df = pd.DataFrame(np.random.randint(1,10, size=(8,3)), columns=list('XYZ'), index=list('abcdefgh'))
df

In [ ]:
df['my str'] = 'text' + df.X.astype(str)
df

In [ ]:
df['mylist'] = np.random.randint(1,10,(8,3)).tolist()
df

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df['Y']   # access a column

In [ ]:
df.Y   # same usng dot notation

In [ ]:
df[['Y', 'my str']]   # access multiple columns

In [ ]:
df.loc['c':'d']   # access rows via the index

In [ ]:
df.loc['c':'d', 'Y']   # access a particlar series via loc of index and column

In [ ]:
df.loc['c', 'Z']   # access a particlar value

In [ ]:
df.loc['c':'d', 'Y':'mylist']   # access a range of rows/cols

In [ ]:
over = df['X'] > 3
over

In [ ]:
df[over]

In [ ]:
df[ df['X'] > 3 ]

In [ ]:
5 in df.mylist

In [ ]:
df.mylist.apply(lambda x: 5 in x)

In [ ]:
df[df['mylist'].apply(lambda x: 5 in x)]

In [ ]:
df[df['mylist'].apply(lambda x: any(filter(lambda y: y > 5, x)))]

In [ ]:
%matplotlib inline 

In [ ]:
df[['X', 'Y', 'Z']].plot()

In [ ]:
df.describe()

In [ ]:
df.describe().transpose()

In [ ]:
df.info()

In [ ]:
df.info(verbose=True, memory_usage='deep')

# Pandas demo:

In [ ]:
!dir AllConf.csv

In [ ]:
from zipfile import ZipFile
import os
import re
from io import StringIO

In [ ]:
def findgrouppattern(grouppattern, haystack):
    """Yield a list of match start, ends and values in haystack"""
    count = 0
    for m in re.finditer(grouppattern, haystack):
        count += 1
        yield (m.start(), m.end(), m.group(1))
    if count > 0:
        yield (len(haystack), 0, '') # Also return end of data
    else:
        yield None
        
def raidconf2dfdict(filename):
    """Return a dictionary of dataframes for each section of AllConf.csv"""
    if filename.lower().endswith('.zip'):
        text = ZipFile(filename).read('AllConf.csv').decode()
    else:
        if os.path.basename(filename).lower() != 'allconf.csv':
            raise RuntimeError('ERROR: Must be an AllConf.csv file')
        with open(filename) as fh:
            text = fh.read()

    pos = list(findgrouppattern('<<(.*)>>\r?\n', text))

    if not pos:
        print(f'ERROR: cannot find config sections in: {filename}')
        return None

    data = dict()
    for i in range(1,len(pos)):
        data[pos[i-1][2]] = pd.read_csv(StringIO(text[pos[i-1][1]:pos[i][0]]), engine='c')
    
    return(data)

In [ ]:
data = raidconf2dfdict('AllConf.csv')

In [ ]:
data.keys()

In [ ]:
data['DKC Information']

In [ ]:
data['Cache']

In [ ]:
data['LDEV'].columns

In [ ]:
data['LDEV']['Volume Size(MB)'].sum() / 1024 / 1024 / 1024

In [ ]:
data['LDEV']['LDEV Attribute'].value_counts()

In [ ]:
data['LDEV']['LDEV Attribute'].value_counts().plot(kind='bar')

In [ ]:
data['LDEV']['LDEV Type'].value_counts()

In [ ]:
mask = data['LDEV']['LDEV Type'] == 'Dynamic Provisioning'
mask

In [ ]:
data['LDEV'][mask]['Volume Size(MB)'].sum() / 1024 / 1024 / 1024

In [ ]:
data['LDEV'][mask]['Current MP Blade'].value_counts().plot(kind='bar')

In [ ]:
data['LDEV'].filter(like='MP Blade').value_counts()